## 1 引入数据集

### 创建数据集

In [9]:
import pandas as pd
from dgl.data import DGLDataset
import torch
import dgl
import os
os.environ["DGLBACKEND"] = "pytorch"

from dataset.RasterDataset import RasterDataset

dataset = RasterDataset(path="./data/")

torch.Size([1152, 35])
torch.Size([1150, 35])
torch.Size([1152, 35])
torch.Size([1152, 35])
torch.Size([1151, 35])
torch.Size([1152, 35])
torch.Size([1151, 35])
torch.Size([1153, 35])
torch.Size([1152, 35])
torch.Size([1151, 35])
torch.Size([1152, 35])
torch.Size([1151, 35])
torch.Size([1151, 35])
torch.Size([1151, 35])
torch.Size([1151, 35])
torch.Size([1151, 35])
torch.Size([1151, 35])
torch.Size([1151, 35])
torch.Size([1152, 35])
torch.Size([1150, 35])
torch.Size([1151, 35])
torch.Size([1152, 35])
torch.Size([1151, 35])
torch.Size([1152, 35])
torch.Size([1152, 35])
torch.Size([1151, 35])
torch.Size([1152, 35])
torch.Size([1153, 35])
torch.Size([1151, 35])
torch.Size([1151, 35])
torch.Size([1150, 35])
torch.Size([1150, 35])
torch.Size([1151, 35])
torch.Size([1150, 35])
torch.Size([1152, 35])
torch.Size([1151, 35])
torch.Size([1151, 35])
torch.Size([1151, 35])
torch.Size([1151, 35])
torch.Size([1151, 35])
torch.Size([1149, 35])
torch.Size([1151, 35])
torch.Size([1149, 35])
torch.Size(

### 查看数据集情况

In [3]:
g = dataset[0]
print(f"Graph of dataset: {g}")
ndata = g.ndata
edata = g.edata
print("Node features")
print(g.ndata)
print("Edge features")
print(g.edata)

Graph of dataset: Graph(num_nodes=1152, num_edges=11930,
      ndata_schemes={'feat': Scheme(shape=(35,), dtype=torch.float64), 'label': Scheme(shape=(2,), dtype=torch.float64), 'train_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool)}
      edata_schemes={'feat': Scheme(shape=(2,), dtype=torch.int64)})
Node features
{'feat': tensor([[ 0.9580,  0.8350,  0.6370,  ...,  1.2120, -0.4660,  0.1170],
        [ 0.5150,  0.5230,  0.2360,  ...,  0.3180, -0.8420, -0.3550],
        [ 3.1720,  3.1140,  2.8000,  ...,  2.4730, -0.6430,  0.2980],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       dtype=torch.float64), 'label': tensor([[72.7780, 80.4900],
        [71.3280, 73.3200],
        [75.0750, 84.2700],
        ...,
        [ 0.00

## 2 创建模型

### 定义模型

In [4]:
import os
os.environ["DGLBACKEND"] = "pytorch"
import dgl
import dgl.data
import torch
import torch.nn as nn
import torch.nn.functional as F

#### 1 GCN

In [5]:
from dgl.nn import GraphConv

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, o_feats):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, h_feats)
        self.conv3 = GraphConv(h_feats, h_feats)
        self.conv4 = GraphConv(h_feats, o_feats)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        h = F.relu(h)
        h = self.conv3(g, h)
        h = F.relu(h)
        h = self.conv4(g, h)
        return h



#### 2 GNN

In [5]:
from dgl.nn import GraphConv

# TODO
class GNN(nn.Module):
    def __init__(self, in_feats, h_feats, o_feats):
        super(GNN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, h_feats)
        self.conv3 = GraphConv(h_feats, h_feats)
        self.conv4 = GraphConv(h_feats, o_feats)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        h = F.relu(h)
        h = self.conv3(g, h)
        h = F.relu(h)
        h = self.conv4(g, h)
        return h


#### 3 GAT

In [7]:
from dgl.nn import GATConv


class GAT(nn.Module):
    def __init__(self, in_feats, h_feats, o_feats, heads):
        super(GAT, self).__init__()
        assert (h_feats % heads == 0)
        h_feats = int(h_feats / heads)
        self.conv1 = GATConv(in_feats, h_feats, heads)
        self.conv2 = GATConv(h_feats*heads, h_feats, heads)
        self.conv3 = GATConv(h_feats*heads, h_feats, heads)
        self.conv4 = GATConv(h_feats*heads, o_feats, 1)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = h.view(h.shape[0], -1)
        h = F.relu(h)

        h = self.conv2(g, h)
        h = h.view(h.shape[0], -1)
        h = F.relu(h)

        h = self.conv3(g, h)
        h = h.view(h.shape[0], -1)
        h = F.relu(h)

        h = self.conv4(g, h)
        h = h.view(h.shape[0], -1)
        return h

#### 4 GraphSAGE

In [57]:
from dgl.nn import SAGEConv

class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats, o_feats,category='pool'):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, category)
        self.conv2 = SAGEConv(h_feats, h_feats, category)
        self.conv3 = SAGEConv(h_feats, h_feats, category)
        self.conv4 = SAGEConv(h_feats, o_feats, category)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        h = F.relu(h)
        h = self.conv3(g, h)
        h = F.relu(h)
        h = self.conv4(g, h)
        return h

#### 5 GIN 

In [8]:
from dgl.nn import GraphConv

# TODO
class GIN(nn.Module):
    def __init__(self, in_feats, h_feats, o_feats):
        super(GIN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, h_feats)
        self.conv3 = GraphConv(h_feats, h_feats)
        self.conv4 = GraphConv(h_feats, o_feats)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        h = F.relu(h)
        h = self.conv3(g, h)
        h = F.relu(h)
        h = self.conv4(g, h)
        return h

#### 6 SGC

In [54]:
from dgl.nn import SGConv

class SGC(nn.Module):
    def __init__(self, in_feats, h_feats, o_feats,k):
        super(SGC, self).__init__()
        self.conv1 = SGConv(in_feats, h_feats, k)
        self.conv2 = SGConv(h_feats, h_feats, k)
        self.conv3 = SGConv(h_feats, h_feats, k)
        self.conv4 = SGConv(h_feats, o_feats, k)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        h = F.relu(h)
        h = self.conv3(g, h)
        h = F.relu(h)
        h = self.conv4(g, h)
        return h

#### 7CNN 8LSTM 9Bi-LSTM  不予考虑

## 3 开始训练

### 定义单图训练函数

In [6]:
def train_single_graph(g, model,num_epochs):
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.01)
    best_val_loss = float('inf')
    best_test_loss = float('inf')

    features = g.ndata["feat"]
    labels = g.ndata["label"]
    train_mask = g.ndata["train_mask"]
    val_mask = g.ndata["val_mask"]
    test_mask = g.ndata["test_mask"]
    for e in range(num_epochs):
        # Forward
        logits = model(g, features)
        # Compute prediction
        pred = logits

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        train_loss = F.mse_loss(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        val_loss = F.mse_loss(logits[val_mask], labels[val_mask])
        test_loss = F.mse_loss(logits[test_mask], labels[test_mask])


        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_loss > val_loss:
            best_val_loss = val_loss
            best_test_loss = test_loss

        # Backward
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print(
                f"In epoch {e}, loss: {train_loss:.3f}, val acc: {val_loss:.3f} (best {best_val_loss:.3f}), test acc: {test_loss:.3f} (best {best_test_loss:.3f})"
            )


### 定义整个训练函数

In [7]:
def train_all_graphs(graphs, model,num_epochs):
    for graph in graphs:
        train_single_graph(graph,model,num_epochs)

### 开始训练

#### 训练1 GCN

In [10]:
# Create the model with given dimensions
model = GCN(g.ndata["feat"].shape[1], 16, g.ndata["label"].shape[1])
train_all_graphs(dataset,model,10)

DGLError: There are 0-in-degree nodes in the graph, output for those nodes will be invalid. This is harmful for some applications, causing silent performance regression. Adding self-loop on the input graph by calling `g = dgl.add_self_loop(g)` will resolve the issue. Setting ``allow_zero_in_degree`` to be `True` when constructing this module will suppress the check and let the code run.

#### 训练2 GNN

In [59]:
# TODO

#### 训练3 GAT

In [12]:
# Create the model with given dimensions
model = GAT(g.ndata["feat"].shape[1], 32, g.ndata["label"].shape[1], heads=16)
train_all_graphs(dataset,model,1000)

In epoch 0, loss: 4766.344, val acc: 4669.621 (best 4669.621), test acc: 4477.388 (best 4477.388)
In epoch 5, loss: 2842.580, val acc: 2753.836 (best 2753.836), test acc: 2696.712 (best 2696.712)
In epoch 10, loss: 287.975, val acc: 429.356 (best 341.194), test acc: 298.733 (best 289.948)
In epoch 15, loss: 303.280, val acc: 432.594 (best 341.194), test acc: 343.236 (best 289.948)
In epoch 20, loss: 317.368, val acc: 342.776 (best 210.071), test acc: 326.608 (best 183.439)
In epoch 25, loss: 184.052, val acc: 188.236 (best 188.236), test acc: 193.499 (best 193.499)
In epoch 30, loss: 198.115, val acc: 191.839 (best 173.417), test acc: 244.794 (best 196.555)
In epoch 35, loss: 153.184, val acc: 172.159 (best 157.790), test acc: 187.873 (best 186.011)
In epoch 40, loss: 131.087, val acc: 157.631 (best 157.631), test acc: 174.192 (best 174.192)
In epoch 45, loss: 125.145, val acc: 165.730 (best 153.800), test acc: 187.002 (best 173.744)
In epoch 50, loss: 109.066, val acc: 158.187 (best 1

#### 训练4 GraphSAGE

In [58]:
# Create the model with given dimensions
model = GraphSAGE(g.ndata["feat"].shape[1], 32, g.ndata["label"].shape[1])
train_all_graphs(dataset,model,10)

torch.Size([1152, 2])
In epoch 0, loss: 5714.539, val acc: 5834.650 (best 5834.650), test acc: 5801.143 (best 5801.143)
torch.Size([1152, 2])
torch.Size([1152, 2])
torch.Size([1152, 2])
torch.Size([1152, 2])
torch.Size([1152, 2])
In epoch 5, loss: 483.357, val acc: 662.589 (best 662.589), test acc: 858.811 (best 858.811)
torch.Size([1152, 2])
torch.Size([1152, 2])
torch.Size([1152, 2])
torch.Size([1152, 2])
torch.Size([1152, 2])
In epoch 0, loss: 266.211, val acc: 221.738 (best 221.738), test acc: 200.968 (best 200.968)
torch.Size([1152, 2])
torch.Size([1152, 2])
torch.Size([1152, 2])
torch.Size([1152, 2])
torch.Size([1152, 2])
In epoch 5, loss: 458.969, val acc: 385.573 (best 221.738), test acc: 307.820 (best 200.968)
torch.Size([1152, 2])
torch.Size([1152, 2])
torch.Size([1152, 2])
torch.Size([1152, 2])
torch.Size([1152, 2])
In epoch 0, loss: 790.900, val acc: 579.517 (best 579.517), test acc: 477.848 (best 477.848)
torch.Size([1152, 2])
torch.Size([1152, 2])
torch.Size([1152, 2])
to

#### 训练5 GIN

In [60]:
# TODO

#### 训练6 SGC

In [56]:
# Create the model with given dimensions
model = SGC(g.ndata["feat"].shape[1], 32, g.ndata["label"].shape[1],k=8)
train_all_graphs(dataset,model,10)

torch.Size([1152, 2])
In epoch 0, loss: 5040.311, val acc: 4952.508 (best 4952.508), test acc: 4734.534 (best 4734.534)
torch.Size([1152, 2])
torch.Size([1152, 2])
torch.Size([1152, 2])
torch.Size([1152, 2])
torch.Size([1152, 2])
In epoch 5, loss: 3833.973, val acc: 3636.981 (best 3636.981), test acc: 3286.644 (best 3286.644)
torch.Size([1152, 2])
torch.Size([1152, 2])
torch.Size([1152, 2])
torch.Size([1152, 2])
torch.Size([1152, 2])
In epoch 0, loss: 731.441, val acc: 534.354 (best 534.354), test acc: 385.034 (best 385.034)
torch.Size([1152, 2])
torch.Size([1152, 2])
torch.Size([1152, 2])
torch.Size([1152, 2])
torch.Size([1152, 2])
In epoch 5, loss: 569.029, val acc: 441.469 (best 441.469), test acc: 397.334 (best 397.334)
torch.Size([1152, 2])
torch.Size([1152, 2])
torch.Size([1152, 2])
torch.Size([1152, 2])
torch.Size([1152, 2])
In epoch 0, loss: 1149.747, val acc: 859.730 (best 859.730), test acc: 538.009 (best 538.009)
torch.Size([1152, 2])
torch.Size([1152, 2])
torch.Size([1152, 

# 4 保存模型参数

In [ ]:
checkpoint_path = "./checkpoints/GAT.pt"
torch.save(model.state_dict(), checkpoint_path)